In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [2]:
installs = pd.read_csv('datos/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
tiempo_max = 60*60*24*3

In [3]:
test_y_temp = installs['2019-04-21':'2019-04-23']
test_y_temp = test_y_temp.reset_index()
test_y_temp['created'] = test_y_temp['created'] - pd.Timestamp(f'2019-04-21 00:00:00')
test_y_temp['created'] = test_y_temp['created'].transform(lambda x: x.total_seconds())
test_y_temp = test_y_temp.groupby('ref_hash').agg({'created' : 'min'}).reset_index()
test_y_temp = test_y_temp[["ref_hash", "created"]]

In [4]:
test_y = pd.read_csv("final.csv", names=["ref_hash"])

In [5]:
hashes = test_y_temp["ref_hash"]
hashes_norm = (hashes-hashes.mean())/hashes.std()
train_x_scaled = hashes_norm
train_y = test_y_temp["created"]

In [6]:
reg = MLPRegressor(verbose=True, random_state=123,
                                          activation="relu",
                                          solver="adam",
                                          alpha=0.01,
                                        max_iter=70)

In [7]:
train_reshaped = np.array(train_x_scaled).reshape(-1,1)

In [8]:
reg.fit(train_reshaped, train_y)

Iteration 1, loss = 11471919694.85833168
Iteration 2, loss = 11455022237.13585281
Iteration 3, loss = 11423427849.14821434
Iteration 4, loss = 11380091983.65438271
Iteration 5, loss = 11326767241.04800606
Iteration 6, loss = 11264545143.74076653
Iteration 7, loss = 11194172464.41510582
Iteration 8, loss = 11116139698.32357979
Iteration 9, loss = 11030837734.05351257
Iteration 10, loss = 10938570506.51630592
Iteration 11, loss = 10839571976.75502777
Iteration 12, loss = 10734028061.07415962
Iteration 13, loss = 10622200645.64143372
Iteration 14, loss = 10504310266.58315468
Iteration 15, loss = 10380543538.42240143
Iteration 16, loss = 10251127971.60030365
Iteration 17, loss = 10116274718.48656654
Iteration 18, loss = 9976240746.04617691
Iteration 19, loss = 9831256447.15257645
Iteration 20, loss = 9681488777.88869667
Iteration 21, loss = 9527246986.50987053
Iteration 22, loss = 9368760674.47177505
Iteration 23, loss = 9206364064.41841888
Iteration 24, loss = 9040185162.85167313
Iteratio

/home/julian/.local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=70, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=123, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [9]:
target_norm = (test_y-test_y.mean())/test_y.std()
target_norm = np.array(target_norm).reshape(-1,1)

In [10]:
predicciones = reg.predict(target_norm)

In [11]:
auxiliar = test_y.copy()
auxiliar["prediccion"] = predicciones

In [12]:
resultados = auxiliar.merge(test_y, how="right").fillna(60*60*24*3)

In [15]:
resultados.columns = ["ref_hash", "obj"]

In [13]:
resultados.to_csv("prediccion_mlp_simple_replica.csv", index=False)